YOLOv5s Android Real-Time Detection Training

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs available:", tf.config.list_physical_devices('GPU'))
print("Is torch cuda available", torch.cuda.is_available()

and

torch.cuda.get_device_name(0)

TensorFlow version: 2.10.0
GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import tensorflow as tf
import platform
import subprocess
import os

tf.random.set_seed(73)
TPU_INIT = False

# Uncomment the following line to disable GPU support
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Configure TensorFlow to use GPU memory efficiently
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"{len(gpus)} GPU(s) found and configured.")
    except RuntimeError as e:
        print("Error configuring GPU memory:", e)
else:
    print("No GPU found by TensorFlow.")

# Check GPU status depending on OS
system_platform = platform.system()
if system_platform in ['Linux', 'Darwin', 'Windows']:
    try:
        result = subprocess.run(["nvidia-smi"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            print("nvidia-smi output:\n", result.stdout)
        else:
            print("nvidia-smi not found or no NVIDIA GPU available.")
    except FileNotFoundError:
        print("nvidia-smi not found. Make sure NVIDIA drivers are installed.")
else:
    print(f"No NVIDIA GPU check supported for platform: {system_platform}")

print("TensorFlow version:", tf.__version__)


1 GPU(s) found and configured.
nvidia-smi output:
 Fri Aug 29 06:16:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.28                 Driver Version: 576.28         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8              4W /   30W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A

In [1]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA devices:", torch.cuda.device_count())

OSError: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\USER\miniconda3\envs\Thesis\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [1]:
import os
import sys
import subprocess
import shutil
from pathlib import Path
import yaml
import torch
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

OSError: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\USER\miniconda3\envs\Thesis\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.